In [1]:

#%%
from ns3gym import ns3env
from comet_ml import Experiment, Optimizer
import tqdm
import subprocess
from collections import deque
import numpy as np

from agents.dqn.agent import Agent, Config
from agents.dqn.model import QNetworkTf
from agents.teacher import Teacher, EnvWrapper
from preprocessor import Preprocessor

/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sheila/anaconda3/envs/environment_py3_7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sheila/

In [2]:
#%%
scenario = "basic"

simTime = 60 # seconds
stepTime = 0.01  # seconds
history_length = 300

EPISODE_COUNT = 15
steps_per_ep = int(simTime/stepTime)

sim_args = {
    "simTime": simTime,
    "envStepTime": stepTime,
    "historyLength": history_length,
    "agentType": Agent.TYPE,
    "scenario": "basic",#"convergence"
    "nWifi": 2, #15
}

print("Steps per episode:", steps_per_ep)

threads_no = 1
env = EnvWrapper(threads_no, **sim_args)

Steps per episode: 6000
Waiting for simulation script to connect on port: tcp://localhost:19944
Please start proper ns-3 simulation script using ./waf --run "..."


In [3]:
#%%
env.reset()
ob_space = env.observation_space
ac_space = env.action_space

print("Observation space shape:", ob_space)
print("Action space shape:", ac_space)

assert ob_space is not None

#%%
teacher0 = Teacher(env, 1, Preprocessor(False))
teacher1 = Teacher(env, 1, Preprocessor(False))

lr = 4e-4
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
config = Config(buffer_size=3*steps_per_ep*threads_no, batch_size=32, gamma=0.7, tau=1e-3, lr=lr, update_every=1)
agent0 = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent0.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

agent1 = Agent(QNetworkTf, history_length, action_size=7, config=config)
agent1.set_epsilon(0.9, 0.001, EPISODE_COUNT-2)

25.745544|0|0|2|-nan
Observation space shape: (1, 300)
Action space shape: (1, 1)




The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.dense instead.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





Action space: 7
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Action space: 7


In [ ]:
# Test the model
hyperparams = {**config.__dict__, **sim_args}
tags = ["Rew: normalized speed",
        "Final",
        f"{Agent.NAME}",
        sim_args['scenario'],
        f"LR: {lr}",
        f"Instances: {threads_no}",
        f"Station count: {sim_args['nWifi']}",
        *[f"{key}: {sim_args[key]}" for key in list(sim_args)[:3]]]
# # agent.save()
logger = teacher0.train(agent0, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)
#logger = teacher.eval(agent,
#                        simTime=simTime,
#                        stepTime=stepTime,
#                        history_length=history_length,
#                        tags=tags,
#                        parameters=hyperparams)
# agent.save()


# %%
logger = teacher1.train(agent1, EPISODE_COUNT,
                        simTime=simTime,
                        stepTime=stepTime,
                        history_length=history_length,
                        send_logs=True,
                        experimental=True,
                        tags=tags,
                        parameters=hyperparams)

COMET INFO: Experiment is live on comet.ml https://www.comet.com/sheila-janota/rl-in-wifi/e32f2348f8ed4e7d85ae7e84fef140eb



0
25.745544|0|0|2|-nan


  5%|▌         | 333/6300 [00:23<09:04, 10.96it/s, curr_speed=34.45 Mbps, mb_sent=11.02 Mb]

------- STARTED TRAINING -------


100%|██████████| 6300/6300 [08:02<00:00, 13.06it/s, curr_speed=34.46 Mbps, mb_sent=2102.97 Mb]


Waiting for simulation script to connect on port: tcp://localhost:44758
Please start proper ns-3 simulation script using ./waf --run "..."
Sent 2102.97 Mb/s.	Mean speed: 35.05 Mb/s	Episode 1/15 finished

1
25.745544|0|0|2|-nan


  1%|          | 66/6300 [00:02<05:21, 19.38it/s, curr_speed=0.00 Mbps, mb_sent=0.00 Mb]COMET ERROR: File could not be uploaded
COMET ERROR: File could not be uploaded
 20%|██        | 1275/6300 [01:37<06:29, 12.91it/s, curr_speed=35.95 Mbps, mb_sent=350.05 Mb]